In [ ]:
# Biblotecas de manipulação de dados
import numpy as np
import pandas as pd
# Bibliotecas de visualização
import seaborn as sns
import matplotlib.pyplot as plt
# Testes estatísticos
from scipy.stats import ttest_ind
# Bibliotecas Locais
from carga import *
from tratamentos import *
from categorizacao import *

pfEnade = doCarga('microdados_enade_2019.txt')
# TRATAMENTOS 
pfEnade = doIdade(pfEnade)
pfEnade = doNota(pfEnade)
pfEnade = doVazios(pfEnade)
pfEnade = doPublicoPrivada(pfEnade)
pfEnade = doHumanasExatas(pfEnade)
pfEnade = doRenda(pfEnade)
pfEnade = doMigracao(pfEnade)
# TRATAMENTOS - Dicionários
pfEnade = doRenda(pfEnade)
#pfEnade['Renda'].value_counts().sort_index()
# pfEnade.isna().sum()    # Mostra se há vazios
pfEnade = doEscolaridade(pfEnade)

In [ ]:
# TABELA - ÚTIL - Incluído
# Renda Familiar X Nota
tmp = pfEnade.groupby('QE_I08')['NT_GER'].agg(['mean', 'std', 'count']).sort_index()
tmp.index = pd.Series(tmp.index).replace(doRendaDIC())
tmp.index.name = 'Qual a renda total de sua família, incluindo seus rendimentos?'
tmp 

In [ ]:
# MAPA CALOR - ÚTIL - Incluído
# CORRELAÇÂO - Renda Familiar e Nota
lista_de_listas = []
valores = sorted(pfEnade['QE_I08'].dropna().unique().tolist())
for i in valores:
    lista_de_listas.append([])
    for j in valores:
        t_res = ttest_ind(pfEnade.query(f'QE_I08 == @i')['NT_GER'], pfEnade.query(f'QE_I08 == @j')['NT_GER'])
        lista_de_listas[-1].append(t_res.pvalue)

valores = pd.Series(valores).replace(doRendaDIC())
tmp = pd.DataFrame(lista_de_listas, index=valores, columns=valores)    

sns.heatmap(tmp, cmap=sns.diverging_palette(150, 275, s=80, l=55, n=9))

In [ ]:
# TABELA - ÚTIL - Incluído
# Horas de Estudo X Renda Familiar
tmp = pfEnade.groupby('QE_I23')['NT_GER'].agg(['mean', 'std', 'count']).sort_index()
tmp.index = pd.Series(tmp.index).replace(doHEstudoDIC())
tmp.index.name = 'Horas de estudo extra classe?'
tmp 

In [ ]:
# MAPA CALOR - ÚTIL - NÂO incluído
# CORRELAÇÂO entre os grupos de horas de estudos
lista_de_listas = []
valores = sorted(pfEnade['QE_I23'].dropna().unique().tolist())
for i in valores:
    lista_de_listas.append([])
    for j in valores:
        t_res = ttest_ind(pfEnade.query(f'QE_I23 == @i')['NT_GER'], pfEnade.query(f'QE_I23 == @j')['NT_GER'])
        lista_de_listas[-1].append(t_res.pvalue)

valores = pd.Series(valores).replace(doHEstudoDIC())
tmp = pd.DataFrame(lista_de_listas, index=valores, columns=valores)    

sns.heatmap(tmp, cmap=sns.diverging_palette(150, 275, s=80, l=55, n=9))

In [ ]:
# TABELA - ÚTIL 
# AGRUPANDO Renda X Horas de Estudo X Nota Média
tmp = pfEnade.groupby(['QE_I08', 'QE_I23'])['NT_GER'].agg(['mean']).unstack()
    
tmp.index = pd.Series(tmp.index).replace(doRendaDIC())
tmp.index.name = 'Qual a renda total de sua família, incluindo seus rendimentos?'

tmp.columns = tmp.columns.droplevel(0)
tmp.columns = pd.Series(tmp.columns).replace(doHEstudoDIC())
tmp.columns.name = 'Quantas horas por semana, aproximadamente, você dedicou aos estudos, excetuando as horas de aula?'
    
tmp 

In [ ]:
# TABELA - Potencial útil
tmp = pfEnade.groupby(['QE_I08', 'QE_I23'])['NT_GER'].agg(['mean']).unstack() #.where(pfEnade.QE_I08 != ' ')
    
#tmp.index = pd.Series(tmp.index).replace(doRendaDIC())
#tmp.index.name = 'Qual a renda total de sua família, incluindo seus rendimentos?'
tmp.index.name = 'Renda total familiar'

tmp.columns = tmp.columns.droplevel(0)

tmp.columns = pd.Series(tmp.columns).replace(doHEstudoDIC())
#tmp.columns.name = 'Quantas horas por semana, aproximadamente, você dedicou aos estudos, excetuando as horas de aula?'
tmp.columns.name = 'Horas por semana extra classe'

    
ax = tmp.plot(figsize=(8, 5)) 
#ax.set_title('Nota X Renda X Horas', fontsize=15)

textoRenda = "Renda Familiar\n"
for key, value in doRendaDIC().items():
    textoRenda = textoRenda + "{k} : {v}\n".format(k=key, v=value)
ax.text(1.01, 0.6, textoRenda, transform=ax.transAxes, fontsize=8)

#textoEstudo = ""
#for key, value in doRendaDIC().items():
#    textoEstudo = textoEstudo + "{k} : {v}\n".format(k=key, v=value)
#ax.legend(textoEstudo)
ax.legend(bbox_to_anchor=(1.01, 0.5), loc='upper left', borderaxespad=0.)

In [ ]:
#MAPA CALOR - Potencial útil
sns.heatmap(data=tmp, cmap=sns.diverging_palette(150, 275, s=80, l=55, n=9), )

In [ ]:
# TABELA - Possívelmente útil
# Gênero X Nota
tmp = pfEnade.groupby('TP_SEXO')['NT_GER'].agg(['mean', 'std', 'count']).sort_index()
tmp.index = pd.Series(tmp.index).replace(doSexoDIC())
tmp.index.name = 'Sexo'
tmp 

In [ ]:
# TABELA - Possívelmente útil
# Gênero X Nota
tmp.div(tmp.iloc[1])

In [ ]:
# TABELA - Utilidade não definida
# Sexo X Renda
SxR = pfEnade.groupby(['TP_SEXO', 'QE_I08'])['NT_GER'].agg(['count']).unstack()
    
SxR.index = pd.Series(SxR.index).replace(doSexoDIC())
SxR.index.name = 'Sexo'

SxR.columns = SxR.columns.droplevel(0)
SxR.columns = pd.Series(SxR.columns).replace(doRendaDIC())
SxR.columns.name = 'Qual a renda total de sua família, incluindo seus rendimentos?'
    
SxR 

In [ ]:
# BUG
fig, ax = plt.subplots(figsize=(16,8))
plt.suptitle('Título Plot')
SxR.filter(items=['TP_SEXO', 'QE_I08', 'NT_GER'])\
    .groupby(['TP_SEXO', 'QE_I08'])\
    .mean().sort_values(by='NT_GER', ascending=False)\
    .unstack().plot(ax=ax)

In [ ]:
SxR.div(SxR.iloc[1])

In [ ]:
# TABELA - Não útil
# Migrado X Renda X Nota
xq = pfEnade.where(pfEnade.Migrado == 1)
#xq = xq.where(xq.NT_GER != ' ')
#xq = xq.where(xq.NT_GER > 0)
#xq = xq.where(xq.QE_I08 != ' ')

MxR = xq.groupby(['NT_GER', 'QE_I08'])['Migrado'].agg(['count']).unstack()
    
MxR.index = pd.Series(MxR.index).replace(doMigradoDIC())
MxR.index.name = 'Migrado'

MxR.columns = MxR.columns.droplevel(0)
MxR.columns = pd.Series(MxR.columns).replace(doRendaDIC())
MxR.columns.name = 'Qual a renda total de sua família, incluindo seus rendimentos?'
    
MxR 

In [ ]:
# DISTRIBUIÇÂO - UTIL - Utilizado
#tmp = pfEnade.groupby(['QE_I08', 'QE_I23'])['NT_GER'].agg(['mean']).unstack() #.where(pfEnade.QE_I08 != ' ')
# MxR = pfEnade.groupby(['NT_GER', 'QE_I08'])['Migrado'].agg(['count']).unstack().where('NT_GER > 0')
#MxR = pfEnade.groupby(['NT_GER', 'QE_I08'])['Migrado'].agg(['count']).unstack().where(pfEnade.NT_GER != ' ')
xq = pfEnade.where(pfEnade.Migrado == 1)
MxR = xq.groupby(['NT_GER',  'RendaDIC'])['Migrado']\
    .agg(['count'])\
    .unstack()
    
MxR.index.name = 'Notas'

MxR.columns = MxR.columns.droplevel(0)
MxR.columns.name = 'Renda'
    
ax = MxR.plot(figsize=(10,5)) 
ax.legend(bbox_to_anchor=(0.7, 1), loc=0, borderaxespad=0.)
#ax.set_title('Nota X Renda X Migração', fontsize=20)

# textoRenda = ""
# for key, value in doRendaDIC().items():
#    textoRenda = textoRenda + "{k} : {v}\n".format(k=key, v=value)
#ax.text(0.05, 0.6, textoRenda, transform=ax.transAxes, fontsize=8)

#textoEstudo = ""
#for key, value in doRendaDIC().items():
#    textoEstudo = textoEstudo + "{k} : {v}\n".format(k=key, v=value)
#ax.legend(textoEstudo)